Paquetes necesarios para el código

In [4]:
import cv2
import mediapipe as mp
import numpy as np

Configuración de Mediapipe y prueba con la webcam

In [5]:
# Inicializamos MediaPipe para detección de rostros
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)  # Usa la cámara predeterminada

while True:
    # Capturamos el fotograma de la webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convertimos el fotograma a RGB (requisito para MediaPipe)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    # Verificamos si se ha detectado alguna cara
    if results.detections:
        for detection in results.detections:
            # Obtenemos la caja delimitadora de la cara
            bboxC = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x, y, width, height = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)

            # Dibujamos el rectángulo alrededor de la cara
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

            # Dibujamos los puntos clave (MediaPipe ofrece 6 puntos clave en detección de rostro)
            for keypoint in detection.location_data.relative_keypoints:
                keypoint_x = int(keypoint.x * w)
                keypoint_y = int(keypoint.y * h)
                cv2.circle(frame, (keypoint_x, keypoint_y), 5, (0, 0, 255), -1)

    # Mostramos el fotograma con las detecciones
    cv2.imshow("MediaPipe Face Detection", frame)
    
    # Para salir, pulsa la tecla "q"
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Liberamos los recursos
cap.release()
cv2.destroyAllWindows()

Creación del filtro del perro de snapchat

In [3]:
# Inicializamos MTCNN para detección de rostros
detector = MTCNN()
cap = cv2.VideoCapture(0) 

# Carga las imágenes PNG con fondo transparente
dog_ears = cv2.imread("assets/dog_ears.png", -1)
dog_nose = cv2.imread("assets/dog_nose.png", -1)
dog_tongue = cv2.imread("assets/dog_tongue.png", -1)

def overlay_image(background, overlay, x, y, w, h):
    overlay = cv2.resize(overlay, (w, h))
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay
    
    for c in range(3):  # Para cada canal de color (BGR)
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detectar cara y puntos clave
    results = detector.detect_faces(frame)

    for result in results:
        keypoints = result["keypoints"]
        x, y, width, height = result["box"]
        
        if y is None:
            continue
        # Poner las orejas sobre la cabeza
        ear_width = width  # Ajustamos el ancho de las orejas al ancho de la cara
        ear_height = int(ear_width * dog_ears.shape[0] / dog_ears.shape[1])  # Mantener proporción de la imagen
        frame = overlay_image(frame, dog_ears, x, y - ear_height // 2, ear_width, ear_height)

        # Poner la nariz sobre la nariz
        nose_width = width // 3
        nose_height = int(nose_width * dog_nose.shape[0] / dog_nose.shape[1])
        frame = overlay_image(frame, dog_nose, keypoints["nose"][0] - nose_width // 2, 
                              keypoints["nose"][1] - nose_height // 2, nose_width, nose_height)

        # Detectar si la boca está abierta y colocar la lengua
        mouth_left = keypoints["mouth_left"]
        mouth_right = keypoints["mouth_right"]
        
        # Calculamos el centro de la boca
        mouth_center_x = (mouth_left[0] + mouth_right[0]) // 2
        mouth_center_y = (mouth_left[1] + mouth_right[1]) // 2

        # Calculamos la distancia entre la nariz y el centro de la boca
        nose_to_mouth_distance = np.linalg.norm(np.array(keypoints["nose"]) - np.array([mouth_center_x, mouth_center_y]))

        # Establecemos un umbral para la apertura de la boca
        mouth_open_threshold = height / 3
        if nose_to_mouth_distance > mouth_open_threshold:
            tongue_width = nose_width  # Usamos el ancho de la nariz para mantener una escala adecuada
            tongue_height = int(tongue_width * dog_tongue.shape[0] / dog_tongue.shape[1])
            frame = overlay_image(frame, dog_tongue, mouth_center_x - tongue_width // 2, 
                                  mouth_center_y, tongue_width, tongue_height)

    # Mostramos el fotograma con el filtro aplicado
    cv2.imshow("Dog Filter with Open Mouth Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

ValueError: operands could not be broadcast together with shapes (68,194) (0,194) 